In [1]:
import pandas as pd

In [4]:
pprecipes = pd.read_csv("../archive/PP_recipes.csv")

In [5]:
rawrecipes = pd.read_csv("../archive/RAW_recipes.csv")

In [7]:
TECHNIQUES_LIST = ['bake', 'barbecue', 'blanch', 'blend', 'boil', 'braise', 'brine', 'broil', 'caramelize', 'combine', 'crock pot', 'crush', 'deglaze', 'devein', 'dice', 'distill', 'drain', 'emulsify', 'ferment', 'freez', 'fry', 'grate', 'griddle', 'grill', 'knead', 'leaven', 'marinate', 'mash', 'melt', 'microwave', 'parboil', 'pickle', 'poach', 'pour', 'pressure cook', 'puree', 'refrigerat', 'roast', 'saute', 'scald', 'scramble', 'shred', 'simmer', 'skillet', 'slow cook', 'smoke', 'smooth', 'soak', 'sous-vide', 'steam', 'stew', 'strain', 'tenderize', 'thicken', 'toast', 'toss', 'whip', 'whisk']

In [8]:
interactions = pd.read_csv("../archive/interactions_test.csv") + pd.read_csv("../archive/interactions_train.csv") + pd.read_csv("../archive/interactions_validation.csv")

In [9]:
rawinteractions = pd.read_csv("../archive/RAW_interactions.csv")

In [65]:
def bayesian_average(r,v,c,m=10): #score function that takes into account the number of reviews
    return (r*v/(v+m))+(c*m/(v+m))

"""
r = Moyenne des notes pour une recette.
v = Nombre d'évaluations pour cet item.
c = Moyenne globale des notes sur l'ensemble du dataset.
m = Seuil minimum de votes requis pour qu'un item soit considéré pertinent (une constante que vous définissez).
"""

"\nr = Moyenne des notes pour une recette.\nv = Nombre d'évaluations pour cet item.\nc = Moyenne globale des notes sur l'ensemble du dataset.\nm = Seuil minimum de votes requis pour qu'un item soit considéré pertinent (une constante que vous définissez).\n"

In [95]:
r = mean_score(40893,rawinteractions)
v = nb_reviews(40893,rawinteractions)
c = global_mean_score(rawinteractions)
bayesian_average(r,v,c,10)

4.4258466998773365

In [51]:
def mean_score(recipe_id, database): 
    return database[database['recipe_id'] == recipe_id]["rating"].mean()

In [54]:
def global_mean_score(database):
    return database["rating"].mean()

In [61]:
def nb_reviews(recipe_id, database):
    return database[database["recipe_id"] == recipe_id].shape[0]

In [118]:
import ast  # ou import json



# Utilisation d'ast.literal_eval pour convertir les strings en listes
pprecipes['techniques'] = pprecipes['techniques'].apply(ast.literal_eval)


In [130]:
# Calculer le total des techniques
# Créer un tableau de totaux initialisé à zéro
total_counts = pd.Series([0] * len(TECHNIQUES_LIST), index=TECHNIQUES_LIST)

# Ajouter les occurrences
for index, row in pprecipes.iterrows():
    for i, value in enumerate(row['techniques']):
        if value == 1:  # Si la technique est présente
            total_counts.at[TECHNIQUES_LIST[i]] += 1

# Afficher les totaux
print("\nFréquence des techniques :")
print(total_counts.sort_values(ascending=False))




Fréquence des techniques :
bake             64821
combine          60136
pour             48747
boil             42354
melt             30865
drain            29567
blend            29387
simmer           29380
skillet          26825
refrigerat       20543
smooth           20465
toss             18166
whisk            16467
saute            15835
fry              15086
thicken          11753
grill             8942
whip              7933
roast             6677
mash              6460
microwave         6405
toast             6259
grate             5584
freez             5405
broil             5178
crush             4737
shred             4702
puree             4535
marinate          4437
dice              4156
steam             3775
strain            3605
soak              3397
knead             2702
crock pot         2628
slow cook         2498
griddle           1248
barbecue          1240
stew              1074
smoke             1023
pickle             984
caramelize         910
scramb

In [129]:
print(total_counts.sort_values(ascending=False))

bake             64821
combine          60136
pour             48747
boil             42354
melt             30865
drain            29567
blend            29387
simmer           29380
skillet          26825
refrigerat       20543
smooth           20465
toss             18166
whisk            16467
saute            15835
fry              15086
thicken          11753
grill             8942
whip              7933
roast             6677
mash              6460
microwave         6405
toast             6259
grate             5584
freez             5405
broil             5178
crush             4737
shred             4702
puree             4535
marinate          4437
dice              4156
steam             3775
strain            3605
soak              3397
knead             2702
crock pot         2628
slow cook         2498
griddle           1248
barbecue          1240
stew              1074
smoke             1023
pickle             984
caramelize         910
scramble           615
deglaze    